In [1]:
#Mount Drive and install dependencies if running in Colab
def install_dependecies():
  !pip install transformers
  !pip install pytorch-lightning

from sys import path
import os
import sys

if 'google.colab' in str(get_ipython()):
  from google.colab import drive

  root_PATH = '/content/drive/My Drive/nlp-seminar/repository'
  drive_mount_location = '/content/drive'
  module_path = root_PATH + '/src'
  
  drive.mount(drive_mount_location, force_remount=True)
  path.append(root_PATH)

  install_dependecies()
else:
  root_PATH = os.path.abspath("../../..")
  module_path = os.path.abspath(os.path.join('../../../src'))

%load_ext autoreload
%autoreload 2

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from data_processor import DataProcessor
from model_evaluator import ModelEvaluator
from models.BERT.bert_custom_dataset import BERTCustomDataset
from models.BERT.my_bert_model import MyBERTModel

import pandas as pd
from torch import cuda
from transformers import BertTokenizer, BertModel, BertConfig
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn as nn
import transformers
import numpy as np
from sklearn import metrics
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from sklearn.metrics import accuracy_score
import pytorch_lightning as pl
from pytorch_lightning import Trainer

In [3]:
# Script configuration
MAX_LEN = 200
TEST_BATCH_SIZE = 32

model_to_use = 'bert-base-cased'
model_path = root_PATH + '/models/deep_learning/BERT/tb_logs_corrected_epoc5_backup/my_BERT_model/version_0/checkpoints/epoch=4.ckpt'

#Model parameters
model_params = {
                'test_batch_size': TEST_BATCH_SIZE
                }
gpus_to_use = [0]

#remove under-represented categories below this treshold
underrepresented_threshold = 3000

In [4]:
#Load datasets
train_df = pd.read_csv(root_PATH + '/data/train.csv')
test_df = pd.read_csv(root_PATH + '/data/test.csv')

#Convert topics column to list
train_df["Topic"] = train_df["Topic"].apply(eval)
test_df["Topic"] = test_df["Topic"].apply(eval)

#Generate boolean masks for our datasets
train_boolean_mask = DataProcessor.obtain_boolean_mask_from_dataset(train_df)
test_boolean_mask = DataProcessor.obtain_boolean_mask_from_dataset(test_df)

#Remove underrepresented topics
underrepresented_topics = DataProcessor.get_underrepresented_topics(train_df,underrepresented_threshold)

train_df, remaining_topics = DataProcessor.remove_topics_from_dataset(train_df,train_boolean_mask,underrepresented_topics)
test_df, _ = DataProcessor.remove_topics_from_dataset(test_df,test_boolean_mask,underrepresented_topics)

#Get boolean masks of our new dataset
test_boolean_mask = test_df.iloc[:,9:]

#Select relevant columns for testing our model
test_df["list"] = test_boolean_mask.values.tolist()

test_df = test_df[["Conversation","list"]].reset_index()

In [5]:
tokenizer = BertTokenizer.from_pretrained(model_to_use)

testing_set = BERTCustomDataset(test_df, tokenizer, MAX_LEN)
data_loader = DataLoader(testing_set, batch_size=TEST_BATCH_SIZE, shuffle= False)

In [7]:
model = MyBERTModel.load_from_checkpoint(model_path, hparams = model_params, training_dataset=None, validation_dataset=None, labels=remaining_topics, model_to_use='bert-base-uncased')

trainer = pl.Trainer(gpus=gpus_to_use,
                             #accelerator='dp',
                             #limit_train_batches=32,
                             #limit_val_batches=32,
                             limit_test_batches=60,
                             #max_epochs=200,
                             #logger=logger,
                             #default_root_dir=logs_path
                             )

trainer.test(ckpt_path=model_path,model=model,test_dataloaders=[data_loader])

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Testing: 100%|██████████| 60/60 [01:09<00:00,  1.23s/it]Total accuracy: 0.7890625
Accuracy per label: {'Satisfied users': 0.9296875, 'Bugs': 0.9854166666666667, 'Design & UX': 0.9911458333333333, 'Dissatisfied users': 0.9760416666666667, 'Performance': 0.99375, 'Use cases': 0.996875, 'Gaming': 0.9979166666666667, 'Feature Requests': 0.99375, 'Complexity': 0.9958333333333333, 'Security & Accounts': 0.946875, 'Update': 0.9479166666666666, 'Pricing': 0.946875, 'Camera & Photos': 0.9473958333333333, 'Video': 0.9989583333333333, 'Customer Support': 0.9963541666666667, 'Notifications & Alerts': 1.0, 'Frequency': 0.9963541666666667, 'Advertising': 0.9994791666666667, 'Payment': 0.9989583333333333, 'Connectivity': 0.9979166666666667, 'Devices': 0.9984375, 'Audio': 0.9989583333333333, 'Sign Up & Login': 1.0, 'Location Services': 0.9994791666666667, 'Privacy': 0.9989583333333333, 'In

[{}]